In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [2]:
df_Denmark= pd.read_csv("df_Denmark.csv" ,encoding='unicode_escape')
df_Denmark

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Denmark,2020/1/1,5792202,88%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Denmark,2020/1/2,5792202,88%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Denmark,2020/1/3,5792202,88%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Denmark,2020/1/4,5792202,88%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Denmark,2020/1/5,5792202,88%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,Denmark,2021/9/24,5792202,88%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007409
698,Denmark,2021/9/25,5792202,88%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007402
699,Denmark,2021/9/26,5792202,88%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007400
700,Denmark,2021/9/27,5792202,88%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007404


In [3]:
#shift 21days mortality
df_Denmark_mortality=df_Denmark["mortality"]
df_Denmark_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22     0.000000
23     0.000000
24     0.000000
25     0.000000
26     0.000000
         ...   
697    0.007409
698    0.007402
699    0.007400
700    0.007404
701    0.000000
Name: mortality, Length: 680, dtype: float64

In [4]:
#删除后21天
df_Denmark.drop(df_Denmark.index[681:702], inplace=True)
df_Denmark

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Denmark,2020/1/1,5792202,88%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Denmark,2020/1/2,5792202,88%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Denmark,2020/1/3,5792202,88%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Denmark,2020/1/4,5792202,88%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Denmark,2020/1/5,5792202,88%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,Denmark,2021/9/3,5792202,88%,55040.0,40085921.0,9.468,6895.551,60705.0,10.442,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007436
677,Denmark,2021/9/4,5792202,88%,34193.0,40120114.0,5.882,6901.433,58806.0,10.116,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007435
678,Denmark,2021/9/5,5792202,88%,43773.0,40163887.0,7.530,6908.963,56823.0,9.775,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007422
679,Denmark,2021/9/6,5792202,88%,59320.0,40223207.0,10.204,6919.167,55340.0,9.520,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007418


In [5]:
df_Denmark1=df_Denmark.join(df_Denmark_mortality,how="left",rsuffix="after21days")
df_Denmark1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days
0,Denmark,2020/1/1,5792202,88%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,Denmark,2020/1/2,5792202,88%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,Denmark,2020/1/3,5792202,88%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,Denmark,2020/1/4,5792202,88%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,Denmark,2020/1/5,5792202,88%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,Denmark,2021/9/3,5792202,88%,55040.0,40085921.0,9.468,6895.551,60705.0,10.442,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007436,0.007436
677,Denmark,2021/9/4,5792202,88%,34193.0,40120114.0,5.882,6901.433,58806.0,10.116,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007435,0.007435
678,Denmark,2021/9/5,5792202,88%,43773.0,40163887.0,7.530,6908.963,56823.0,9.775,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007422,0.007422
679,Denmark,2021/9/6,5792202,88%,59320.0,40223207.0,10.204,6919.167,55340.0,9.520,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.007418,0.007418


In [6]:
df_Denmark1.to_csv("df_Denmark_after21days",index=False)